In [2]:
# === Import packages ===
import pandas as pd
import numpy as np
import statsmodels.api as sm

import sys, site
sys.path.append(site.getusersitepackages())

# === Column names from Excel file ===
COL_NONDURABLES = "real consumption of nondurables"
COL_SERVICES    = "real consumption of services"
COL_POP         = "population"

# === 1) Load the Excel file ===
df = pd.read_excel("PS4data.xls")

# === 2) Construct per-capita consumption ===
df["C_pc"] = (df[COL_NONDURABLES] + df[COL_SERVICES]) / df[COL_POP]

# === 3) Set a time index ===
if "year" in df.columns and "quarter" in df.columns:
    df["time"] = df["year"].astype(str) + "-Q" + df["quarter"].astype(str)
    df = df.set_index("time")

# === 4) Create lags C_{t-1} to C_{t-4} ===
for L in range(1, 5):
    df[f"C_pc_L{L}"] = df["C_pc"].shift(L)

# Drop rows with missing lags
df_reg = df.dropna(subset=[f"C_pc_L{L}" for L in range(1, 5)] + ["C_pc"])

# === 5) Run the regression C_t on its lags ===
y = df_reg["C_pc"]
X = df_reg[[f"C_pc_L{L}" for L in range(1, 5)]]
X = sm.add_constant(X)
ols = sm.OLS(y, X).fit()
print(ols.summary())

# === 6) Joint F-test: H0: β2 = β3 = β4 = 0 ===
R = np.array([
    [0, 0, 1, 0, 0],  # β2 = 0
    [0, 0, 0, 1, 0],  # β3 = 0
    [0, 0, 0, 0, 1]   # β4 = 0
])
f_test = ols.f_test(R)
print("\nJoint F-test H0: β2 = β3 = β4 = 0")
print(f"F = {float(f_test.fvalue):.3f},  p-value = {float(f_test.pvalue):.4g}")


                            OLS Regression Results                            
Dep. Variable:                   C_pc   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.094e+05
Date:                Mon, 17 Nov 2025   Prob (F-statistic):               0.00
Time:                        11:05:08   Log-Likelihood:                 1745.1
No. Observations:                 216   AIC:                            -3480.
Df Residuals:                     211   BIC:                            -3463.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.439e-05   1.92e-05      1.267      0.2

In [7]:
# === Part (c): 2SLS in LOGS with White (robust) SEs ===
# Model: Δlog C_t = γ0 + γ1 Δlog Y_t + u_t
# Instruments: log(C_{t-2}/C_{t-3}), log(C_{t-3}/C_{t-4}), log(C_{t-4}/C_{t-5}), log(C_{t-5}/C_{t-6})
%pip install linearmodels
import numpy as np
import pandas as pd
from linearmodels.iv import IV2SLS
import statsmodels.api as sm

# 0) Build real per-capita income
if "Y_pc" not in df.columns:
    df["Y_pc"] = df["real disposable income"] / df["population"]

# 1) Logs
for col in ["C_pc", "Y_pc"]:
    if (df[col] <= 0).any():
        raise ValueError(f"Column {col} has nonpositive values; cannot take log.")
df["logC"] = np.log(df["C_pc"])
df["logY"] = np.log(df["Y_pc"])

# 2) Log-differences (growth rates)
df["gC"] = df["logC"].diff()
df["gY"] = df["logY"].diff()

# 3) Instruments
df["z1"] = df["gC"].shift(2)  # log(C_{t-2}/C_{t-3})
df["z2"] = df["gC"].shift(3)  # log(C_{t-3}/C_{t-4})
df["z3"] = df["gC"].shift(4)  # log(C_{t-4}/C_{t-5})
df["z4"] = df["gC"].shift(5)  # log(C_{t-5}/C_{t-6})

# 4) Collect regression dataset and drop missing rows
vars_needed = ["gC", "gY", "z1", "z2", "z3", "z4"]
data_c = df[vars_needed].dropna().copy()

# 5) 2SLS with White (heteroskedasticity-robust) SEs
iv_c = IV2SLS.from_formula("gC ~ 1 + [gY ~ z1 + z2 + z3 + z4]", data=data_c)\
             .fit(cov_type="robust")
print("=== (c) 2SLS: Δlog C_t on Δlog Y_t (White-robust SEs) ===")
print(iv_c.summary)

# 6) Weak-instrument diagnostics (first stage)
# linearmodels provides per-endogenous first-stage; print F-stat for relevance
fs = iv_c.first_stage                   # FirstStageResults object
print("\n--- First-stage for Δlog Y_t ---")
print(fs.summary)

# Excluded-instruments F-stat (relevance)
try:
    print(f"First-stage excluded instruments F: {fs.f_stat.stat:.3f}, p = {fs.f_stat.pval:.4g}")
except AttributeError:
    pass  # older versions may not have f_stat

# 7) Overidentification test (with >1 instrument): J/Sargan test
# (With robust covariance, linearmodels reports a robust J-statistic.)
if hasattr(iv_c, "j_stat") and iv_c.j_stat is not None:
    J = iv_c.j_stat.stat
    J_p = iv_c.j_stat.pval
    J_df = iv_c.j_stat.df
    print(f"\nOveridentification (J) test: J = {J:.3f}, df = {J_df}, p = {J_p:.4g}")


=== (c) 2SLS: Δlog C_t on Δlog Y_t (White-robust SEs) ===
                          IV-2SLS Estimation Summary                          
Dep. Variable:                     gC   R-squared:                      0.0685
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0641
No. Observations:                 214   F-statistic:                    4.6529
Date:                Sun, Nov 16 2025   P-value (F-stat)                0.0310
Time:                        21:17:06   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.0026     0.0012     2.2579     0.0240      0.0003      0

In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import warnings

# --- reuse data_d and iv_c if present---
need_build = False
for name in ["data_d"]:
    if name not in globals():
        need_build = True
        break

if need_build:

    df["logC"] = np.log(df["C_pc"])
    df["logY"] = np.log(df["real disposable income"] / df["population"])
    df["gC"] = df["logC"].diff()
    df["gY"] = df["logY"].diff()
    df["z1"] = df["gC"].shift(2)
    df["z2"] = df["gC"].shift(3)
    df["z3"] = df["gC"].shift(4)
    df["z4"] = df["gC"].shift(5)
    vars_needed = ["gC","gY","z1","z2","z3","z4"]
    data_d = df[vars_needed].dropna().copy()

from linearmodels.iv import IV2SLS
iv_c = IV2SLS.from_formula("gC ~ 1 + [gY ~ z1 + z2 + z3 + z4]", data=data_d).fit(cov_type="robust")
print("Reprinted 2SLS with robust SEs:\n")
print(iv_c.summary)

# ---------- (1) Control-function endogeneity test (DWH) ----------
Z_excl = data_d[["z1","z2","z3","z4"]]
Z1 = sm.add_constant(Z_excl)                     # instruments + constant
fs = sm.OLS(data_d["gY"], Z1).fit()
data_d["vhat"] = fs.resid
X_cf = sm.add_constant(pd.concat([data_d["gY"], data_d["vhat"]], axis=1))
ols_cf = sm.OLS(data_d["gC"], X_cf).fit(cov_type="HC1")
print("\n=== Endogeneity test (control-function) ===")
print(ols_cf.summary())
# silence future warning by asking for a scalar explicitly
with warnings.catch_warnings():
    warnings.simplefilter("ignore", FutureWarning)
    print("\nTest H0: vhat = 0  (exogeneity)")
    print(ols_cf.wald_test("vhat = 0", scalar=True))

# ---------- (2) Robust Hansen J manual (White/EHW) ----------

e = iv_c.resids.values.reshape(-1,1)            # 2SLS residuals
Z = Z_excl.values                                # excluded instruments (n x L), L=4
n, L = Z.shape
g_bar = (Z * e).mean(axis=0).reshape(L,1)        # (1/n) Σ z_i e_i
# EHW (White) covariance of moments: S = (1/n) Σ (e_i^2 z_i z_i')
S = sum((e[i,0]**2) * np.outer(Z[i,:], Z[i,:]) for i in range(n)) / n
J_robust = n * float(g_bar.T @ np.linalg.inv(S) @ g_bar)
df_j = L - 1   # one endogenous regressor
from scipy import stats
p_robust = 1 - stats.chi2.cdf(J_robust, df_j)
print(f"\n=== Robust Hansen J (manual): J = {J_robust:.3f}, df = {df_j}, p = {p_robust:.4g}")

# ---------- (3) Homoskedastic Sargan (just for reference) ----------
# Sargan uses homoskedastic weighting
# X includes const and endogenous regressor
X = sm.add_constant(data_d[["gY"]]).values
M_X = np.eye(n) - X @ np.linalg.inv(X.T @ X) @ X.T
sigma2 = float((e.T @ e) / (n - X.shape[1]))
Sargan = float((e.T @ Z @ np.linalg.inv(Z.T @ M_X @ Z) @ Z.T @ e) / sigma2)
p_sargan = 1 - stats.chi2.cdf(Sargan, df_j)
print(f"Sargan (homoskedastic): J = {Sargan:.3f}, df = {df_j}, p = {p_sargan:.4g}")


Reprinted 2SLS with robust SEs:

                          IV-2SLS Estimation Summary                          
Dep. Variable:                     gC   R-squared:                      0.0685
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0641
No. Observations:                 214   F-statistic:                    4.6529
Date:                Mon, Nov 17 2025   P-value (F-stat)                0.0310
Time:                        11:05:22   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.0026     0.0012     2.2579     0.0240      0.0003      0.0049
gY             0.43